In [2]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv,find_dotenv
from IPython.display import display_markdown
from openai import OpenAI


load_dotenv(find_dotenv(),override=True)

client = OpenAI()

In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

In [6]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="gpt-4.1-nano"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [7]:
display_markdown(mergesort_code, raw=True)

Certainly! Here's a clear and efficient Python implementation of the Merge Sort algorithm:

```python
def merge_sort(array):
    """
    Sorts the input array using the Merge Sort algorithm.

    Args:
        array (list): The list of elements to sort.

    Returns:
        list: A new sorted list containing the elements from the input array.
    """
    if len(array) <= 1:
        return array

    # Find the middle point and split the array
    mid = len(array) // 2
    left_half = merge_sort(array[:mid])
    right_half = merge_sort(array[mid:])

    return merge(left_half, right_half)

def merge(left, right):
    """
    Merges two sorted lists into a single sorted list.

    Args:
        left (list): First sorted list.
        right (list): Second sorted list.

    Returns:
        list: Merged and sorted list.
    """
    merged = []
    i = j = 0

    # Traverse both lists and insert the smaller element from either list into merged
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            merged.append(left[i])
            i += 1
        else:
            merged.append(right[j])
            j += 1

    # Collect the remaining elements (if any)
    merged.extend(left[i:])
    merged.extend(right[j:])

    return merged

# Example usage:
if __name__ == "__main__":
    unsorted_list = [38, 27, 43, 3, 9, 82, 10]
    print("Unsorted list:", unsorted_list)
    sorted_list = merge_sort(unsorted_list)
    print("Sorted list:", sorted_list)
```

### Explanation:
- `merge_sort` recursively divides the list into halves until it reaches lists of length 1 or 0.
- The `merge` function then combines two sorted lists into one sorted list.
- The example demonstrates how to use the merge sort function with a sample list.

Feel free to ask if you'd like additional features or explanations!

### Reflection step

In [8]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

In [10]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="gpt-4.1-nano"
).choices[0].message.content

In [11]:
display_markdown(critique, raw=True)


Great job on implementing a clear, efficient, and Pythonic merge sort! Your code adheres to best practices, including proper documentation, recursive structure, and efficient merging.

Here are some points of critique and suggestions that might enhance your implementation:

### Strengths:
- **Clarity & Readability:** Well-structured code with meaningful variable names and comments.
- **Efficiency:** The merge operation is implemented optimally, leveraging list extension for remaining elements.
- **Functional Approach:** Your implementation treats merge_sort as a pure function, returning new sorted lists, which is idiomatic in Python.

---

### Suggestions for Improvement:

1. **In-Place Sorting:**
   - If memory efficiency is critical, consider implementing an in-place merge sort (though it’s more complex). The current implementation copies sublists during recursion, which is fine for clarity but may be less memory-efficient for large datasets.

2. **Optimization with Slicing:**
   - Slicing (`array[:mid]` and `array[mid:]`) creates new lists at each recursion, which can be costly for large datasets.
   - If performance is vital, consider using indices to avoid slicing:
     ```python
     def merge_sort(array, start=0, end=None):
         if end is None:
             end = len(array)
         # base case
         if end - start <= 1:
             return array[start:end]
         mid = (start + end) // 2
         left_half = merge_sort(array, start, mid)
         right_half = merge_sort(array, mid, end)
         return merge(left_half, right_half)
     ```
   - However, this complicates the interface and may not be necessary unless performance profiling indicates bottlenecks.

3. **Handling Custom Comparators:**
   - For enhanced flexibility, you could add an optional `key` or `cmp` parameter, similar to `sorted()`:
     ```python
     def merge_sort(array, key=None):
         # Implementation that uses key for comparisons
     ```
   - This allows sorting based on object attributes or custom criteria.

4. **Testing and Edge Cases:**
   - Include more comprehensive tests, e.g., empty list, single-element list, list with duplicate elements, already sorted list, reverse-sorted list.

5. **Type Hints:**
   - To leverage Python's static analysis tools, consider adding type hints:
     ```python
     from typing import List, TypeVar

     T = TypeVar('T')

     def merge_sort(array: List[T]) -> List[T]:
         ...
     ```
   - This aids readability and catches type errors early.

---

### Final Note:
Your implementation strikes a good balance between simplicity and correctness. The suggestions above are mainly for optimization or extending functionality. If your use case involves large datasets or complex objects, consider the optional enhancements.

If you'd like, I can help you implement variants like in-place merge sort, parallel merging, or comparison-based customizations.

In [12]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)


### Generation Step (II)


In [13]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="gpt-4.1-nano"
).choices[0].message.content

In [14]:
display_markdown(essay, raw=True)

Thank you for the detailed feedback and constructive suggestions! Based on your points, here's a revised version incorporating some of your recommendations:

- Added type hints for better static analysis.
- Implemented an optional `key` parameter to support custom comparison criteria.
- Used indices to avoid slicing for improved performance.
- Provided additional test cases, including edge cases.

Here's the improved implementation:

```python
from typing import List, TypeVar, Optional

T = TypeVar('T')

def merge_sort(array: List[T], key: Optional[callable] = None) -> List[T]:
    """
    Performs a merge sort on the input list, optionally using a key function for comparisons.
    The function is stable and has O(n log n) complexity.

    Args:
        array (List[T]): The list of elements to sort.
        key (Optional[callable]): Optional; a function that extracts a comparison key from each list element.

    Returns:
        List[T]: A new sorted list.
    """
    def compare(x, y):
        if key is None:
            return x <= y
        else:
            return key(x) <= key(y)

    def merge(left: List[T], right: List[T]) -> List[T]:
        merged: List[T] = []
        i = j = 0
        while i < len(left) and j < len(right):
            if compare(left[i], right[j]):
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                j += 1
        merged.extend(left[i:])
        merged.extend(right[j:])
        return merged

    def recursive_sort(start: int, end: int) -> List[T]:
        if end - start <= 1:
            return array[start:end]
        mid = (start + end) // 2
        left_half = recursive_sort(start, mid)
        right_half = recursive_sort(mid, end)
        return merge(left_half, right_half)

    return recursive_sort(0, len(array))


# Example usage and tests:
if __name__ == "__main__":
    test_lists = [
        [],  # Empty list
        [5],  # Single element
        [3, 1, 4, 1, 5, 9, 2, 6],  # Multiple elements with duplicates
        [10, 9, 8, 7],  # Reverse sorted
        [1, 2, 3, 4, 5],  # Already sorted
        ["banana", "apple", "cherry"],  # Strings
    ]

    print("Testing merge_sort with various lists:\n")
    for lst in test_lists:
        print(f"Original: {lst}")
        print(f"Sorted:   {merge_sort(lst)}\n")

    # Testing with custom key (sorting by length)
    words = ["python", "java", "c", "javascript"]
    print(f"Original words: {words}")
    print(f"Sorted by length: {merge_sort(words, key=len)}\n")
```

### What's new:
- **Type hints**: Clarify input/output types.
- **Custom key parameter**: Enables sorting by arbitrary criteria (e.g., string length, object attributes).
- **Efficient index passing**: The main recursive function avoids slicing, reducing memory overhead.
- **More comprehensive tests**: Cover edge cases and demonstrate flexibility with sorting strings and using keys.

Would you like further enhancements, such as in-place sorting, parallel processing, or support for custom comparison functions?

### Implementing a class

In [18]:
#from agentic_patterns import ReflectionAgent
from agents import Agent,Runner

In [21]:
#agent = ReflectionAgent(model="gpt-4.1-nano")



In [23]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [26]:
coder_agent = Agent(
    name="coder_agent",
    model="gpt-4.1-nano",
    instructions=generation_system_prompt
    )


In [27]:
    result = await Runner.run(coder_agent, user_msg)
    print(result.final_output)


Certainly! Here's a Python implementation of the **Merge Sort** algorithm:

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    :param arr: List of elements to be sorted
    :return: New sorted list
    """
    if len(arr) <= 1:
        return arr

    # Find the middle point and divide the array into halves
    mid = len(arr) // 2
    left_half = merge_sort(arr[:mid])
    right_half = merge_sort(arr[mid:])

    # Merge the sorted halves
    return merge(left_half, right_half)

def merge(left, right):
    """
    Merges two sorted lists into a single sorted list.

    :param left: Sorted list
    :param right: Sorted list
    :return: Merged sorted list
    """
    result = []
    i = j = 0

    # Traverse both lists and insert the smaller element from left or right into result
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.appen

In [28]:
reflection_agent = Agent(
    name="reflection_agent",
    model="gpt-4.1-nano",
    instructions=reflection_system_prompt
    )


In [29]:
reflection_result = await Runner.run(coder_agent, result.final_output)
print(reflection_result.final_output)

Your implementation of merge sort looks clean, efficient, and well-commented. It follows the classic recursive approach, making it easy to understand and maintain.

If you'd like to enhance the code further, here are a few suggestions:

1. **In-place Sorting**: Modify the code so that it sorts the list in place, which can be more space-efficient.
2. **Add Type Hints**: Include Python type hints for better readability and static analysis.
3. **Flexibility for Custom Comparison**: Allow passing a custom comparison function if sorting non-standard objects.
4. **Handling Large Data**: For very large datasets, consider an iterative version or techniques to minimize memory usage.

Here's an enhanced version incorporating some of these ideas:

```python
from typing import List, Callable, TypeVar

T = TypeVar('T')

def merge_sort(arr: List[T], compare: Callable[[T, T], bool] = lambda a, b: a <= b) -> None:
    """
    Sorts the list in place using merge sort algorithm.

    :param arr: List of